In [1]:
%%capture
%pip install accelerate peft bitsandbytes transformers trl

In [2]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.6/289.6 kB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.7 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset

med_dataset = "Thaweewat/thai-med-pack"

In [4]:
huggingface_dataset = load_dataset(med_dataset, split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/189190 [00:00<?, ? examples/s]

In [5]:
huggingface_dataset

Dataset({
    features: ['text'],
    num_rows: 189190
})

In [6]:
import pandas as pd

# การสุ่มตัวอย่าง 10% ของข้อมูลโดยไม่ให้มีการหยิบซ้ำ
dataset_pd = pd.DataFrame(huggingface_dataset).sample(frac=0.1, replace=False)

In [7]:
dataset_pd.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18919 entries, 177365 to 153289
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    18919 non-null  object
dtypes: object(1)
memory usage: 295.6+ KB


In [8]:
from datasets import Dataset

# สมมุติว่า df เป็น DataFrame ที่คุณได้แก้ไขแล้ว
customized_dataset = Dataset.from_pandas(dataset_pd)

In [9]:
def transform_structure(example):
    # Extract and split the text
    instruction_text = example['text'].split('[INST] ')[1].split(' [/INST]')[0]
    response_text = example['text'].split('[INST] ')[1].split(' [/INST]')[1].strip()

    # Strip off the ending token </s> if it's present
    if response_text.endswith('</s>'):
        response_text = response_text[:-4].strip()

    # Return the new structure
    return {
        'instruction': instruction_text,
        'input': '',
        'output': response_text,
        'text': ''
    }

# Apply the transformation to each record in the dataset again
transformed_dataset = customized_dataset.map(transform_structure)


Map:   0%|          | 0/18919 [00:00<?, ? examples/s]

In [10]:
customized_dataset[18000]

{'text': '<s>[INST] ลิ้นหัวใจรั่ว แต่ผ่าตัดเปลี่ยนลิ้นหัวใจแล้ว แล้วหัวใจจะหายเป็นปกติเหมือนเดิมไหมค่ะ [/INST] สวัสดีค่ะ การที่ลิ้นหัวใจรั่ว หากได้รับการเปลี่ยนลิ้นหัวใจ และไม่มีภาวะแทรกซ้อนจากการผ่าตัด อาการมักดีขึ้นค่ะแต่ทั้งนี้ขึ้นกับปัจจัยต่างๆ ที่ส่งผลต่อการรักษา เช่น โรคประจำตัวหรือโรคร่วมอื่น, ความสม่ำเสมอในการรับประทานยา และการติดตามการรักษา เป็นต้นในรายละเอียดการรักษา รวมถึงตัวโรคที่เป็นอยู่ แนะนำ สอบถามจากแพทย์ที่ดูแลอีกทีนะคะ </s>',
 '__index_level_0__': 45034}

In [11]:
transformed_dataset[18000]

{'text': '',
 '__index_level_0__': 45034,
 'instruction': 'ลิ้นหัวใจรั่ว แต่ผ่าตัดเปลี่ยนลิ้นหัวใจแล้ว แล้วหัวใจจะหายเป็นปกติเหมือนเดิมไหมค่ะ',
 'input': '',
 'output': 'สวัสดีค่ะ การที่ลิ้นหัวใจรั่ว หากได้รับการเปลี่ยนลิ้นหัวใจ และไม่มีภาวะแทรกซ้อนจากการผ่าตัด อาการมักดีขึ้นค่ะแต่ทั้งนี้ขึ้นกับปัจจัยต่างๆ ที่ส่งผลต่อการรักษา เช่น โรคประจำตัวหรือโรคร่วมอื่น, ความสม่ำเสมอในการรับประทานยา และการติดตามการรักษา เป็นต้นในรายละเอียดการรักษา รวมถึงตัวโรคที่เป็นอยู่ แนะนำ สอบถามจากแพทย์ที่ดูแลอีกทีนะคะ'}

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "openthaigpt/openthaigpt-1.0.0-beta-13b-chat-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


pytorch_model.bin.index.json:   0%|          | 0.00/29.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.99G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.96G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/6.27G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/1.82k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/718k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.47M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/469 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/170 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "self_attn.q_proj",
        "self_attn.k_proj",
        "self_attn.v_proj",
        "self_attn.o_proj",
        # Add other relevant submodules from LlamaDecoderLayer here
    ]
)

In [16]:
# ตัวอย่างการกำหนดขนาดข้อมูลการฝึก
per_device_train_batch_size = 2 ## จากเดิม 4
gradient_accumulation_steps = 4 ## จากเดิม 2
total_train_data_size = len(transformed_dataset)  # จำนวนข้อมูลฝึกทั้งหมด
batch_size = per_device_train_batch_size * 1  # ถ้ามี 1 GPU
steps_per_epoch = int(total_train_data_size / (batch_size * gradient_accumulation_steps))
desired_epochs = 1  # ตัวอย่างจำนวน epoch ที่ต้องการ

# คำนวณ max_steps ที่จะตั้งใน TrainingArguments
max_steps_calculated = steps_per_epoch * desired_epochs

In [17]:
max_steps_calculated

2364

In [18]:
from transformers import TrainingArguments

output_dir = "/content/drive/MyDrive/llm_ThaiMed_checkpoints/thai-med-pack"
gradient_accumulation_steps = gradient_accumulation_steps
optim = "paged_adamw_32bit"
save_steps = 50
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 100
warmup_ratio = 0.03
lr_scheduler_type = "cosine"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps_calculated,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [19]:
def formatting_prompts_func(examples):
    output_text = []
    for i in range(len(examples)):
        instruction = examples["instruction"][i]
        response = examples["output"][i]
        text = f'<s>[INST] <<SYS>You are a question answering assistant. Answer the question as truthful and helpful as possible คุณคือผู้ช่วยตอบคำถาม จงตอบคำถามอย่างถูกต้องและมีประโยชน์ที่สุด<</SYS>>{instruction}[/INST]{response}'
        output_text.append(text)
    return output_text

In [20]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(41088, 5120)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (v_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (o_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (up_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear4bit(in_features=13824, out_features=5120, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Lla

In [21]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=transformed_dataset,
    peft_config=peft_config,
    formatting_func=formatting_prompts_func,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:269: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConf

Map:   0%|          | 0/18919 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:397: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


In [22]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [23]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
10,3.400500
20,3.274600
30,2.997400
40,2.793500
50,2.644800
60,2.437600
70,2.309400
80,2.160800
90,1.943000
100,1.640000


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in

TrainOutput(global_step=2364, training_loss=0.1589512423834371, metrics={'train_runtime': 7027.2404, 'train_samples_per_second': 2.691, 'train_steps_per_second': 0.336, 'total_flos': 4.693161724432896e+17, 'train_loss': 0.1589512423834371, 'epoch': 197.0})

In [24]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(41088, 5120)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=5120, out_features=5120, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=5120, out_features=64, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=64, out_features=5120, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=5120, out_features=5120, bias=False)
            (lora_dropout): ModuleDict(
    

# **Save to Huggingface.co**

In [25]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "openthaigpt/openthaigpt-1.0.0-beta-13b-chat-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [26]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [28]:


custom_prompt = "โปรดอธิบายลักษณะช่องปากที่เป็นมะเร็งในระยะเริ่มต้น"


PROMPT =f'[INST] <You are a question answering assistant. Answer the question as truthful and helpful as possible คุณคือผู้ช่วยตอบคำถาม จงตอบคำถามอย่างถูกต้องและมีประโยชน์ที่สุด<>{custom_prompt}[/INST]'



In [29]:
%%time
from peft import PeftModel
# from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig
from transformers import GenerationConfig

model = PeftModel.from_pretrained(model, "/content/drive/MyDrive/llm_ThaiMed_checkpoints/thai-med-pack/checkpoint-2350")

inputs = tokenizer(
    PROMPT,
    return_tensors="pt",
)
input_ids = inputs["input_ids"].cuda()

generation_config = GenerationConfig(
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.15,
)
print("Generating...")
generation_output = model.generate(
    input_ids=input_ids,
    generation_config=generation_config,
    return_dict_in_generate=True,
    output_scores=True,
    max_new_tokens=256,
    pad_token_id=tokenizer.eos_token_id
)
for s in generation_output.sequences:
    print(tokenizer.decode(s))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configurati

Generating...
<s> [INST] <You are a question answering assistant. Answer the question as truthful and helpful as possible คุณคือผู้ช่วยตอบคำถาม จงตอบคำถามอย่างถูกต้องและมีประโยชน์ที่สุด<>โปรดอธิบายลักษณะช่องปากที่เป็นมะเร็งในระยะเริ่มต้น[/INST]ในขั้นต้น มะเร็งช่องปากจะแสดงอาการเป็นแผลหรือตุ่มที่ลิ้น เหงือก เพดานปาก หรือคอหอย อาการอื่นๆ อาจรวมถึงความเจ็บปวด การบวม และการสูญเสียฟัน</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></

In [30]:
model.save_pretrained('finetunedMedmodelv2')
tokenizer.save_pretrained('finetunedMedmodelv2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


('finetunedMedmodelv2/tokenizer_config.json',
 'finetunedMedmodelv2/special_tokens_map.json',
 'finetunedMedmodelv2/tokenizer.model',
 'finetunedMedmodelv2/added_tokens.json',
 'finetunedMedmodelv2/tokenizer.json')

In [33]:
model.push_to_hub('openthaigpt-MedChatModelv2')
tokenizer.push_to_hub('openthaigpt-MedChatModelv2')

adapter_model.safetensors:   0%|          | 0.00/419M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/718k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Aekanun/openthaigpt-MedChatModelv2/commit/46d29f7c026c217aa1373fdc26db9c32a0c253e4', commit_message='Upload tokenizer', commit_description='', oid='46d29f7c026c217aa1373fdc26db9c32a0c253e4', pr_url=None, pr_revision=None, pr_num=None)

In [34]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PeftModelForCausalLM(
      (base_model): LoraModel(
        (model): LlamaForCausalLM(
          (model): LlamaModel(
            (embed_tokens): Embedding(41088, 5120)
            (layers): ModuleList(
              (0-39): 40 x LlamaDecoderLayer(
                (self_attn): LlamaSdpaAttention(
                  (q_proj): lora.Linear4bit(
                    (base_layer): Linear4bit(in_features=5120, out_features=5120, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=5120, out_features=64, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=64, out_features=5120, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
         